In [1]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)

In [2]:
L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x, y, z, yaw, pitch, roll, q1, q2, q3 = sp.symbols('x_b y_b z_b yaw pitch roll q_1 q_2 q_3', real=True)

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

In [3]:
# ROTATIONS
# Intrinsic ZYX
Rz = sp.Matrix([
    [sp.cos(yaw), -sp.sin(yaw), 0],
    [sp.sin(yaw),  sp.cos(yaw), 0],
    [0,            0,           1]
])

Ry = sp.Matrix([
    [sp.cos(pitch),  0, sp.sin(pitch)],
    [0,              1, 0            ],
    [-sp.sin(pitch), 0, sp.cos(pitch)]
])

Rx = sp.Matrix([
    [1, 0,            0           ],
    [0, sp.cos(roll), -sp.sin(roll)],
    [0, sp.sin(roll),  sp.cos(roll)]
])

# Intrinsic ZYX: Postmultiplication
R_zyx_intrinsic = Rz * Ry * Rx

# Extrinsic XYZ: Premultiplication
R_xyz_extrinsic = Rz * Ry * Rx

sp.pprint(R_xyz_extrinsic)
sp.pprint(R_zyx_intrinsic)

⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)                     sin(roll)⋅cos(pitch)                      ↪

↪ pitch)⋅cos(roll)⋅cos(yaw) + sin(roll)⋅sin(yaw)⎤
↪                                               ⎥
↪ pitch)⋅sin(yaw)⋅cos(roll) - sin(roll)⋅cos(yaw)⎥
↪                                               ⎥
↪            cos(pitch)⋅cos(roll)               ⎦
⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)    

### Conventions
For arm 1, the nominal configuration is straight out to the right of the body (i.e. along positive $Y_{b}$). For arm 2, the nominal configuration is straight out to the left of the body (i.e. along the negative $Y_{b}$).

Rotations around relative axes are intrinsic and thus you use postmultiplication of the rotation.

In [11]:
# Body frame b: FRD at vehicle CoM
T_Ib = sp.Matrix([[sp.cos(yaw)*sp.cos(pitch), sp.cos(yaw)*sp.sin(pitch)*sp.sin(roll)-sp.sin(yaw)*sp.cos(roll), sp.cos(yaw)*sp.sin(pitch)*sp.cos(roll)+sp.sin(yaw)*sp.sin(roll), x],
                  [sp.sin(yaw)*sp.cos(pitch), sp.sin(yaw)*sp.sin(pitch)*sp.sin(roll)+sp.cos(yaw)*sp.cos(roll), sp.sin(yaw)*sp.sin(pitch)*sp.cos(roll)-sp.cos(yaw)*sp.sin(roll), y],
                  [-sp.sin(pitch), sp.cos(pitch)*sp.sin(roll), sp.cos(pitch)*sp.cos(roll), z],
                  [0, 0, 0, 1]])

# Frame 0: Z axis of rotation pointing forward, X axis pointing up, Y axis pointing right (90 deg rotation around Yb)
T_b0 = sp.Matrix([[0, 0, 1, 0],
                  [0, 1, 0, 0],
                  [-1, 0, 0, 0],
                  [0, 0, 0, 1]])

# Frame 1: Rotation around Z0 by q1
T_01 = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0, 0],
                [sp.sin(q1),  sp.cos(q1), 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

# Frame 2: Translation along Y1 of L1, then rotation around Y1 by pi/2 and rotation around new Z2 by q2
rot_y1 = sp.Matrix([[sp.cos(sp.pi/2), 0, sp.sin(sp.pi/2)],
                    [0, 1, 0],
                    [-sp.sin(sp.pi/2), 0, sp.cos(sp.pi/2)]])
rot_z2 = sp.Matrix([[sp.cos(q2), -sp.sin(q2), 0],
                    [sp.sin(q2), sp.cos(q2), 0],
                    [0, 0, 1]])
rot_y1z2 = rot_y1 * rot_z2 # postmultiply because z2 is relative axis
T_12 = sp.diag(rot_y1z2, 1)
T_12[0, 3] = L1

# Frame 3: Translation along Y2 by L2, rotation about Y2 by -pi/2, then rotation about new Z3 by q3
rot_y2 = sp.Matrix([[sp.cos(-sp.pi/2), 0, sp.sin(-sp.pi/2)],
                    [0, 1, 0],
                    [-sp.sin(-sp.pi/2), 0, sp.cos(-sp.pi/2)]])
rot_z3 = sp.Matrix([[sp.cos(q3), -sp.sin(q3), 0],
                    [sp.sin(q3), sp.cos(q3), 0],
                    [0, 0, 1]])
rot_y2z3 = rot_y2 * rot_z3 # postmultiply because z3 is relative axis
T_23 = sp.diag(rot_y2z3, 1)
T_23[0, 3] = L2

# Frame e: Translation along Y3 by L3, rotation about X3 by -pi/2
T_3e = sp.Matrix([[1, 0, 0, 0],
                [0, 0, 1, L3],
                [0, -1, 0, 0],
                [0, 0, 0, 1]])

T_be = sp.trigsimp(sp.expand(T_b0*T_01*T_12*T_23*T_3e))
    
# Set parameters
T_be = T_be.subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

In [12]:
# Forward kinematics function
FK = sp.Matrix([T_be[0,3],
                T_be[1,3],
                T_be[2,3],
                sp.atan2(T_be[2,1], T_be[2,2]),
                sp.asin(T_be[0,3]),
                sp.atan2(T_be[1,0], T_be[0,0])])

sp.pprint("State dependencies")
print(f"xe = f({FK[0,0].free_symbols})")
print(f"ye = f({FK[1,0].free_symbols})")
print(f"ze = f({FK[2,0].free_symbols})")
print(f"yawe = f({FK[3,0].free_symbols})")
print(f"rolle = f({FK[4,0].free_symbols})")
print(f"pitche = f({FK[5,0].free_symbols})")



State dependencies
xe = f({q_2, q_3})
ye = f({q_3, q_1, q_2})
ze = f({q_1, q_3, q_2})
yawe = f({q_3, q_1, q_2})
rolle = f({q_2, q_3})
pitche = f({q_1, q_3, q_2})


In [13]:
# Forward kinematics function
T_Ie = T_Ib * T_be
FK_centralised = sp.Matrix([T_Ie[0,3],
                T_Ie[1,3],
                T_Ie[2,3],
                sp.atan2(T_Ie[2,1], T_Ie[2,2]),
                sp.asin(T_Ie[0,3]),
                sp.atan2(T_Ie[1,0], T_Ie[0,0])])

sp.pprint("State dependencies")
print(f"xe = f({FK_centralised[0,0].free_symbols})")
print(f"ye = f({FK_centralised[1,0].free_symbols})")
print(f"ze = f({FK_centralised[2,0].free_symbols})")
print(f"yawe = f({FK_centralised[3,0].free_symbols})")
print(f"rolle = f({FK_centralised[4,0].free_symbols})")
print(f"pitche = f({FK_centralised[5,0].free_symbols})")

State dependencies
xe = f({pitch, q_3, q_2, roll, q_1, x_b, yaw})
ye = f({pitch, q_2, q_3, roll, q_1, yaw, y_b})
ze = f({pitch, q_2, q_3, roll, q_1, z_b})
yawe = f({pitch, q_2, q_3, roll, q_1})
rolle = f({roll, pitch, q_1, q_3, x_b, yaw, q_2})
pitche = f({pitch, q_3, q_2, roll, q_1, yaw})


### Controllability
Since the states roll and y_b, and pitch and x_b are coupled, not the entire end-effector configuration space is reachable.

In [ ]:
# Jacobian calculation for velocity
J = FK_centralised.jacobian([x, y, z, yaw, pitch, roll, q1, q2, q3])

print(J.shape)
J_inv = J.pinv() # use pseudoinverse because J is non-square

(6, 9)
